## Import libraries

In [2]:
import pandas as pd
import numpy as np

from network_optimizer import NetworkOptimizer
import synthetic_data_generators as sdg

## Create Synthetic Data

In [3]:
NETWORK_SIZE = 100
POOL_SIZE = 100
MEMBER_SIZE = 100
NUM_ROUNDS = 5
MIN_LATITUDE = 42000000 #42.015438
MAX_LATITUDE = 42500000 #42.481365
MIN_LONGITUDE = -83600000 #-83.566956
MAX_LONGITUDE = -82600000 #-82.628043

In [4]:
synth_network = sdg.generate_providers(NETWORK_SIZE, MIN_LATITUDE, MAX_LATITUDE, MIN_LONGITUDE, MAX_LONGITUDE)
synth_pool = sdg.generate_providers(POOL_SIZE, MIN_LATITUDE, MAX_LATITUDE, MIN_LONGITUDE, MAX_LONGITUDE)
synth_members = sdg.generate_members(MEMBER_SIZE, MIN_LATITUDE, MAX_LATITUDE, MIN_LONGITUDE, MAX_LONGITUDE)
synth_reqs = sdg.synth_reqs

In [5]:
print(f"Network size: {len(synth_network)}")
print(f"Pool size: {len(synth_pool)}")
print(f"Number of members: {len(synth_members)}")
synth_reqs

Network size: 100
Pool size: 100
Number of members: 100


,specialty,county,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,15,90,5
1,pcp,wayne,15,90,10
2,ent,wayne,15,90,5
3,urologist,wayne,15,90,5
4,obgyn,wayne,15,90,5


## Run Optimization

In [6]:
def user_objective(self, network):
    return self.adequacy(network) / len(network)

In [7]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             network=synth_network,
                             user_objective=user_objective
                             )

In [12]:
optimizer.optimize(5)

Optimization round 6 ...
Optimization round 7 ...
Optimization round 8 ...
Optimization round 9 ...
Optimization round 10 ...
Average seconds per round of optimization: 0.1
Adequacy score for best network: 1



In [13]:
optimizer.performance_history[-10:].round(5)

array([0.0101 , 0.0102 , 0.01031, 0.01042, 0.01053, 0.01064, 0.01075,
       0.01087, 0.01099, 0.01111])

In [14]:
print(f"Adequacy Index: {optimizer.adequacy(optimizer.best_network)}")
print(f"Network size: {len(optimizer.best_network)}")
optimizer.adequacy_detail

Adequacy Index: 1
Network size: 90


,specialty,county,distance_req,min_access_pct,min_providers,members_with_access,pct_members_with_access,provider_count,pct_req_providers
0,cardiologist,wayne,15,90,5,100,1.00,20,4.0
1,pcp,wayne,15,90,10,100,1.00,19,1.9
2,ent,wayne,15,90,5,92,0.92,14,2.8
3,urologist,wayne,15,90,5,99,0.99,18,3.6
4,obgyn,wayne,15,90,5,100,1.00,19,3.8


In [15]:
optimizer.move_tracker

[('removal', 56),
 ('removal', 78),
 ('removal', 18),
 ('removal', 99),
 ('removal', 20),
 ('removal', 82),
 ('removal', 37),
 ('removal', 39),
 ('removal', 4),
 ('removal', 5)]